In [1]:
import os
import pandas as pd

filename = "ValeursFoncieres-2024.txt"
file_path = os.path.join(os.getcwd(), "..", "data", filename)

In [2]:
# On charge le fichier dans un dataframe pandas
col_to_keep = [
    'Date mutation',
    'Nature mutation',
    'Valeur fonciere',
    'No voie',
    'B/T/Q',
    'Type de voie',
    'Code voie',
    'Voie',
    'Code postal',
    'Commune',
    'Code departement',
    'Code commune',
    'Prefixe de section',
    'Section',
    'No plan',
    'Code type local',
    'Type local',
    'Surface reelle bati',
    'Nombre pieces principales',
    'Surface terrain'
]
col_type = {
    'Nature mutation' : 'object',
    'Valeur fonciere' : 'float32',
    'No voie' : 'object',
    'B/T/Q' : 'object',
    'Type de voie' : 'object',
    'Code voie' : 'object',
    'Voie' : 'object',
    'Code postal' : 'object',
    'Commune' : 'object',
    'Code departement' : 'object',
    'Code commune' : 'object',
    'Prefixe de section' : 'object',
    'Section' : 'object',
    'No plan' : 'Int16',
    'Code type local' : 'Int8',
    'Type local' : 'object',
    'Surface reelle bati' : 'Int32',
    'Nombre pieces principales' : 'Int8',
    'Surface terrain' : 'Int32'
}
df = pd.read_csv(
    file_path,
    sep='|',
    decimal=',',
    usecols=col_to_keep,
    dtype=col_type,
    parse_dates=['Date mutation'],
    na_values=['NULL', '']
    )

# Quelques infos générales dont la taille 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3458643 entries, 0 to 3458642
Data columns (total 20 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Date mutation              object 
 1   Nature mutation            object 
 2   Valeur fonciere            float32
 3   No voie                    object 
 4   B/T/Q                      object 
 5   Type de voie               object 
 6   Code voie                  object 
 7   Voie                       object 
 8   Code postal                object 
 9   Commune                    object 
 10  Code departement           object 
 11  Code commune               object 
 12  Prefixe de section         object 
 13  Section                    object 
 14  No plan                    Int16  
 15  Code type local            Int8   
 16  Type local                 object 
 17  Surface reelle bati        Int32  
 18  Nombre pieces principales  Int8   
 19  Surface terrain            Int32  
dtypes:

In [5]:
# Les échanges ne nous intéressent pas ici
print(f"Nb lignes avant nettoyage : {df.shape}")
df = df[df['Nature mutation'] != 'Echange']
print(f"Nb lignes après nettoyage : {df.shape}")

Nb lignes avant nettoyage : (3458643, 20)
Nb lignes après nettoyage : (3410882, 20)


In [6]:
# on élimine les lignes dont les valeurs foncieres sont manquantes
print(len(df))
df = df[df['Valeur fonciere'].notna()]
print(len(df))

3410882
3378011


In [7]:
df['Valeur fonciere'] = df['Valeur fonciere'].astype('int32')

In [8]:
print(f"\nNombre de lignes en double avant suppression : {df.duplicated().sum()}")

# Supprimer les doublons (conserver la première occurrence)
df = df.drop_duplicates()

print(f"Nombre de lignes en double après suppression : {df.duplicated().sum()}")
print(f"Nombre de lignes après suppression des doublons : {df.shape[0]}")


Nombre de lignes en double avant suppression : 485649
Nombre de lignes en double après suppression : 0
Nombre de lignes après suppression des doublons : 2892362


In [9]:
# On va compléter les Nan des Surfaces et nombre de pieces par 0
df.loc[:, 'Surface reelle bati'] = df['Surface reelle bati'].fillna(0)
df.loc[:, 'Surface terrain'] = df['Surface terrain'].fillna(0)
df.loc[:, 'Nombre pieces principales'] = df['Nombre pieces principales'].fillna(0)

# De même sur la colonne B/T/Q par un champ vide
df.loc[:, 'B/T/Q'] = df['B/T/Q'].fillna('')
df.loc[:, 'No voie'] = df['No voie'].fillna('')
df.loc[:, 'Type de voie'] = df['Type de voie'].fillna('')
df.loc[:, 'Voie'] = df['Voie'].fillna('')
df.loc[:, 'Code postal'] = df['Code postal'].fillna('')
df.loc[:, 'Prefixe de section'] = df['Prefixe de section'].fillna('')
df.loc[:, 'Section'] = df['Section'].fillna('')

In [10]:
# On a constaté dans le notebook que e prix d'un terrain étant insignifiant dans le prix d'un bien immobilier,
# nous allons supprimer les lignes : 
#     dont le type de local est Nan car ils n'ont pas de surface bati 
#     dont le type de local est Dépendances car ils n'ont que des que des surfaces de terrain.
#     et les lignes dont la surface reelle bati est nulle
# Ce choix est propre au projet qui ne concernera que les investissmeent en surface bati soit :  maisons, appartements et locaux industriels 
print("nb lignes avant : ", df.shape[0])
filtre = (df['Type local'].isna()) | (df['Type local'] == 'Dépendance') 
df = df[~filtre]
print("nb lignes apres : ", df.shape[0])

nb lignes avant :  2892362
nb lignes apres :  1030477


In [11]:
df.shape[0]

1030477

In [12]:
# On regroupe les transactions par date et valeurs et on crée un id_transaction pour chacun de ces couples
df.loc[:, 'Date mutation'] = pd.to_datetime(df['Date mutation'], format='%d/%m/%Y', errors='coerce') # on s'assure que toutes les dates soient au même format pour le tri, c'est mieux !
df = df.sort_values(by=['Date mutation', 'Valeur fonciere', 'Code departement', 'Code commune']).reset_index(drop=True) 
col_id_transaction = df.groupby(['Date mutation', 'Valeur fonciere', 'Code departement', 'Code commune', 'Code voie']).ngroup()
df.insert(loc=0, column='id_transaction', value=col_id_transaction)

In [13]:

df.head()

,id_transaction,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,Code departement,Code commune,Prefixe de section,Section,No plan,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
0,0,2024-01-01 00:00:00,Vente,1,57,,RUE,1725,EMILE MONTEGUT,87000,...,87,85,,IN,365,1,Maison,91,4,4795
1,0,2024-01-01 00:00:00,Vente,1,39,,RUE,1725,EMILE MONTEGUT,87000,...,87,85,,IN,366,4,Local industriel. commercial ou assimilé,3958,0,6677
2,1,2024-01-02 00:00:00,Vente,500,5113,,,B106,LE PIN,43230,...,43,100,,AL,272,1,Maison,70,2,115
3,2,2024-01-02 00:00:00,Vente,1666,209,,RUE,0120,DES FORTS,88160,...,88,468,,E,208,4,Local industriel. commercial ou assimilé,165,0,0
4,3,2024-01-02 00:00:00,Vente,2500,26,,RUE,0330,DU DOC ROUX,3600,...,03,82,,AE,281,1,Maison,28,2,60


In [14]:
df.describe()

,id_transaction,Valeur fonciere,No plan,Code type local,Surface reelle bati,Nombre pieces principales,Surface terrain
count,1.030477e+06,1.030477e+06,1030477.0,1030477.0,1030477.0,1030477.0,1030477.0
mean,4.156373e+05,5.144428e+05,371.11616,1.660042,119.603091,3.171954,734.339376
std,2.398522e+05,3.406559e+06,544.588033,0.876225,892.524517,1.82674,7722.613831
min,0.000000e+00,0.000000e+00,1.0,1.0,0.0,-58.0,0.0
25%,2.073040e+05,1.165000e+05,79.0,1.0,50.0,2.0,0.0
50%,4.153990e+05,1.929000e+05,197.0,1.0,77.0,3.0,195.0
75%,6.242830e+05,3.190000e+05,440.0,2.0,106.0,4.0,606.0
max,8.274560e+05,2.550000e+08,8395.0,4.0,265000.0,73.0,2960000.0


In [15]:
df.isna().sum()

id_transaction               0
Date mutation                0
Nature mutation              0
Valeur fonciere              0
No voie                      0
B/T/Q                        0
Type de voie                 0
Code voie                    0
Voie                         0
Code postal                  0
Commune                      0
Code departement             0
Code commune                 0
Prefixe de section           0
Section                      0
No plan                      0
Code type local              0
Type local                   0
Surface reelle bati          0
Nombre pieces principales    0
Surface terrain              0
dtype: int64

In [16]:
# On ne garde que les transactions avec une ligne unique car il y a souvent les memes biens qui apparaissent 2 fois sur la même transaction
# Sans doute dû à des corrections dans les champs.
id_transaction_count = df['id_transaction'].value_counts()
unique_id_transactions_values = id_transaction_count[id_transaction_count == 1].index
unique_id_transactions_list = unique_id_transactions_values.tolist()

df =  df[df['id_transaction'].isin(unique_id_transactions_list)]

In [17]:
df.shape

(695669, 21)

In [18]:
# On efface les lignes dont la valeur fonciere est inférieure à 1€ car ce n'est pas représentatif.
# On pourra sans doute augmenter cette valeur autour de 1000€ ou plus
filtre = df['Valeur fonciere'] <= 1
print("Nombre de valeur fonciere <= 1€ : ", df[filtre].shape[0])
df = df[~filtre]
print(len(df))

Nombre de valeur fonciere <= 1€ :  1109
694560


In [19]:
# Et on retire les lignes dont la surface reelle bati est nulle car seul le bati nous interesse
df = df[df['Surface reelle bati'] != 0]
print(len(df))

692448


In [20]:
output_file = "dvf_2024_cleaned.csv"
file_path = os.path.join(os.getcwd(), "..", "data", "01_DVF", output_file)
print(file_path)
df.to_csv(file_path, index=False, sep=';')

/Users/cyriljeanneau/Documents/02. Professionnel/00. Formation Dev IA/Projets/certificationB1/notebooks/../data/01_DVF/dvf_2024_cleaned.csv


In [21]:
df[df['Code postal'] == '37000']['Valeur fonciere'] / df[df['Code postal'] == '37000']['Surface reelle bati']

101        1136.363636
1050       3238.095238
1415       1869.565217
3297       5294.117647
3571       3343.283582
              ...     
1026127    2575.757576
1028503         3000.0
1029034    2962.962963
1029169    2032.967033
1029845     991.735537
Length: 1219, dtype: Float64

In [3]:
for index, row in df.iterrows():

SyntaxError: incomplete input (2006401529.py, line 1)